```
NLP에서 사전 훈련(pre-training)
  Word2Vec, FastText ,GloVe 와 같은 기법은 단어의 고정된 벡터로 변환하여 의미를 학습
  문제점 : 문맥을 고려하지 못해 다의어나 동음이의어 구분이 어렵다
  해결 : 문맥을 반영하는 사전 훈련된 언어 모델이 등장
  2015년 구글이 LSTM 기반 언어모델 사전학습모델로 전이학습을 수행하는 방법제안->
  ELMo(Embedding From Language Model) 순방향, 역방향언어모델을 따로 학습해서 문맥을 반영 임베딩을 생성->
  RNN 에서 Transformer 기반 모델로 전환-> OpenAI GPT-1 등장

```


```
마스크드언어 MLM(Maksed Language Model )
  문장에서 일부 단어를 마스킹하고 이를 예측하도록 하는  학습
  양방향 문맥을 활용할 수 있도록 고안됨
  대표적인 모델이 BERT(Bidirectional Encoder Representations from Transformers) 이후로 NLP 태스크의 성능향상이 이루어졌다

```

# Hugging Face의 ransformers 라이브러를 활용

In [ ]:
!pip install transformers sentencepiece -q

In [ ]:
from transformers import pipeline

In [ ]:
# BERT기반 마스크드 언어 모델 로드
fill_mask = pipeline('fill-mask',model='bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# 예제문장
sentence = 'I want to [MASK] a new book'

In [ ]:
# 모델 실행
predictions = fill_mask(sentence)

In [ ]:
# 예측된 단어 출력
for prediction in predictions:
  print(f'단어 : {prediction["token_str"]} 확률 : {prediction["score"]:.4f}')

단어 : write 확률 : 0.7492
단어 : start 확률 : 0.0681
단어 : buy 확률 : 0.0217
단어 : open 확률 : 0.0171
단어 : make 확률 : 0.0164


# BERT 사전 훈련
```
1. 마스트드 언어모델 MLM(Masked Language Model)
2. 다음문장예측 NSP(Next Stentence Prediction)
```

## 마스크드 언어모델
```
BERT는 사전 훈련을 위해 입력 문장에서 일부 단어를 랜덤하게 [MASK] 토큰으로 바꾼 후, 이 가려진 단어를 예측하는 방식으로 학습됩니다. 이를 **마스크드 언어 모델(Masked Language Model, MLM)**이라고 합니다.
I love machine learning
I love [MASK] learning.
모델은 문맥을 활용하여 [MASK]가 machine임을 예측해야 합니다.

MLM의 학습 과정에서 15%의 단어가 마스킹됩니다. 그러나 단어를 완전히 [MASK]로 대체하면 훈련과 실제 사용 간 차이가 발생할 수 있으므로, 다음과 같은 변형이 적용됩니다.

80%: [MASK]로 대체 (I love [MASK] learning.)
10%: 다른 랜덤 단어로 대체 (I love coffee learning.)
10%: 원래 단어 유지 (I love machine learning.)

이러한 방식은 모델이 [MASK] 토큰에 과도하게 의존하지 않도록 하고, 일반적인 문맥 이해 능력을 향상시킵니다.
```

## 다음문장 예측
```
BERT는 문장 간 관계를 학습하기 위해 **다음 문장 예측(Next Sentence Prediction, NSP)**을 사용합니다. 이는 문장 쌍이 주어졌을 때, 두 번째 문장이 첫 번째 문장의 실제 다음 문장인지 여부를 예측하는 작업입니다.
예를 들어, 다음과 같은 문장이 있다고 가정합니다.
문장 A: "I went to the store."
문장 B: "I bought some milk."
이 경우 문장 B는 문장 A의 다음 문장이므로 라벨은 **IsNext**가 됩니다.
다른 경우로, 문장 A와 B가 연관성이 없는 문장이라면 **NotNext**로 벨링됩니다.

문장 A: "I went to the store."
문장 B: "The weather is nice today."
훈련 데이터의 50%는 실제 다음 문장이고, 나머지 50%는 랜덤한 문장을 조합하여 NSP 태스크를 수행하도록 구성됩니다.
```

## BERT
```
사전 훈련된 BERT 모델을 다양한 NLP 태스크에 적용하려면
파인 튜닝(Fine-tuning)**이 필요합니다. 파인 튜닝 과정에서는 BERT 위에 특정
태스크를 위한 추가 레이어를 쌓고, 전체 네트워크를 도메인에 맞게 재학습합니다.

예를 들어, 문장 분류 태스크를 수행하려면 BERT의 출력인 [CLS] 토큰의 벡터를
활용하여 분류기를 추가하고, 레이블 데이터로 모델을 훈련합니다.
```

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

In [ ]:
# Bert모델 및 토크나이져 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
# 입력데이터 예제
inputs = tokenizer('Hello, my dog is cute',return_tensors='pt')
labels = torch.tensor([1]).unsqueeze(0)  # 긍정 라벨(1)
labels

tensor([[1]])

In [ ]:
# 모델 학습 준비
optimizer = AdamW(model.parameters(),lr=5e-5)
output = model(**inputs,labels=labels)
loss = output.loss
loss.backward()
optimizer.step()

# 데이터셋 정의 하고 실제 학습

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# 데이터 셋 정의
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=self.max_len, return_tensors='pt')
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
# 토크나이저 및 데이터 준비
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
texts = ["I love machine learning!", "This movie was terrible.", "Great experience at the restaurant."]
labels = [1, 0, 1]  # 긍정(1), 부정(0) 레이블

In [ ]:
train_dataset = CustomDataset(texts, labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [ ]:
# BERT 모델 로드 및 학습 설정
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 학습 루프
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

Epoch 1, Loss: 0.2397228106856346
Epoch 2, Loss: 0.18323517590761185
Epoch 3, Loss: 0.13348475843667984
Epoch 4, Loss: 0.08866898901760578
Epoch 5, Loss: 0.07910462561994791
Epoch 6, Loss: 0.09077958017587662
Epoch 7, Loss: 0.050986845046281815
Epoch 8, Loss: 0.03584899473935366
Epoch 9, Loss: 0.03750061662867665
Epoch 10, Loss: 0.03037046268582344
Epoch 11, Loss: 0.024835819378495216
Epoch 12, Loss: 0.013826766517013311
Epoch 13, Loss: 0.016849165316671133
Epoch 14, Loss: 0.01476075779646635
Epoch 15, Loss: 0.010625516530126333
Epoch 16, Loss: 0.009297520155087113
Epoch 17, Loss: 0.006893103709444404
Epoch 18, Loss: 0.006190489744767547
Epoch 19, Loss: 0.006344285677187145
Epoch 20, Loss: 0.004399689845740795
Epoch 21, Loss: 0.004226644290611148
Epoch 22, Loss: 0.005078323185443878
Epoch 23, Loss: 0.005609183572232723
Epoch 24, Loss: 0.0030715852044522762
Epoch 25, Loss: 0.0036762801464647055
Epoch 26, Loss: 0.0033388122683390975
Epoch 27, Loss: 0.00316941412165761
Epoch 28, Loss: 0.0

In [ ]:
# 예측 수행
model.eval()
test_texts = ["I hate spam emails.", "I enjoy deep learning.",'''I fell asleep while watching this movie. When I woke up, it was over. ''']
test_dataset = CustomDataset(test_texts, [0, 1,0], tokenizer)
test_loader = DataLoader(test_dataset, batch_size=1)

with torch.no_grad():
    for batch in test_loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
        print(f"Predicted Label: {predictions.item()}")

Predicted Label: 0
Predicted Label: 1
Predicted Label: 0


# 동일한 모델로 분류시행

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [ ]:
# 모델 및 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# 예측 함수 정의
def predict_sentiment(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Positive" if prediction == 1 else "Negative"


In [ ]:
# 예측 실행
test_sentences = [
    "I love machine learning!",
    "This movie was terrible.",
    "What a wonderful day!",
    "I hate spam emails."
]


In [ ]:
for sentence in test_sentences:
    print(f"Sentence: {sentence} -> Prediction: {predict_sentiment(sentence)}")

Sentence: I love machine learning! -> Prediction: Positive
Sentence: This movie was terrible. -> Prediction: Positive
Sentence: What a wonderful day! -> Prediction: Positive
Sentence: I hate spam emails. -> Prediction: Positive


# Google Bert의 Masked 언어모델

In [ ]:
# 모델과 토크나이져 로딩
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [ ]:
# Bert 모델과 토크나이져 로드
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
from transformers import AutoTokenizer

# BERT 모델에 맞는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

# 두 문장을 연결하여 입력
text = "Soccer is a really fun [MASK]. [SEP] I love playing sports."

# 텍스트를 토크나이저로 변환
inputs = tokenizer(text, return_tensors='tf')

# 결과 출력
print("input_ids:", inputs['input_ids'])  # 정수 인코딩 결과
print("token_type_ids:", inputs['token_type_ids'])  # 세그먼트 인코딩
print("attention_mask:", inputs['attention_mask'])  # 어텐션 마스크


input_ids: tf.Tensor(
[[ 101 4715 2003 1037 2428 4569  103 1012  102 1045 2293 2652 2998 1012
   102]], shape=(1, 15), dtype=int32)
token_type_ids: tf.Tensor([[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 15), dtype=int32)
attention_mask: tf.Tensor([[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 15), dtype=int32)


In [ ]:
# [MASK] 토큰 예측
from transformers import FillMaskPipeline

# FillMaskPipeline 초기화
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

# 예시 문장으로 예측 결과 확인
print(pip('Soccer is a really fun [MASK].'))
print(pip('The Avengers is a really fun [MASK].'))
print(pip('I went to [MASK] this morning.'))

Device set to use 0


[{'score': 0.7621113061904907, 'token': 4368, 'token_str': 'sport', 'sequence': 'soccer is a really fun sport.'}, {'score': 0.20342056453227997, 'token': 2208, 'token_str': 'game', 'sequence': 'soccer is a really fun game.'}, {'score': 0.012208598665893078, 'token': 2518, 'token_str': 'thing', 'sequence': 'soccer is a really fun thing.'}, {'score': 0.001863026642240584, 'token': 4023, 'token_str': 'activity', 'sequence': 'soccer is a really fun activity.'}, {'score': 0.001335486420430243, 'token': 2492, 'token_str': 'field', 'sequence': 'soccer is a really fun field.'}]
[{'score': 0.25628983974456787, 'token': 2265, 'token_str': 'show', 'sequence': 'the avengers is a really fun show.'}, {'score': 0.1728411316871643, 'token': 3185, 'token_str': 'movie', 'sequence': 'the avengers is a really fun movie.'}, {'score': 0.111077219247818, 'token': 2466, 'token_str': 'story', 'sequence': 'the avengers is a really fun story.'}, {'score': 0.07248973101377487, 'token': 2186, 'token_str': 'series'

# 한국어 BERT

In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [ ]:
# BERT 모델과 토크나이져 로드
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

# 정수 인코딩 결과 확인
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [ ]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [ ]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [ ]:
from transformers import FillMaskPipeline

# FillMaskPipeline 생성
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

# 예측 결과 출력
print(pip('영화 오징어 게임은 정말 재미있는  [MASK]다.'))

Device set to use 0


[{'score': 0.48234984278678894, 'token': 3771, 'token_str': '영화', 'sequence': '영화 오징어 게임은 정말 재미있는 영화 다.'}, {'score': 0.10738483816385269, 'token': 568, 'token_str': '거', 'sequence': '영화 오징어 게임은 정말 재미있는 거 다.'}, {'score': 0.04528239741921425, 'token': 3758, 'token_str': '이야기', 'sequence': '영화 오징어 게임은 정말 재미있는 이야기 다.'}, {'score': 0.026212716475129128, 'token': 3949, 'token_str': '친구', 'sequence': '영화 오징어 게임은 정말 재미있는 친구 다.'}, {'score': 0.02503294311463833, 'token': 4697, 'token_str': '재미', 'sequence': '영화 오징어 게임은 정말 재미있는 재미 다.'}]


# 한국어 BERT NSP

In [ ]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

# 모델과 토크나이저 로드
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [ ]:
# 이어지는 문장 예시
# 이어지는 문장은 0  이어지지 않은 문장은 1
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."

# 두 문장을 토크나이저로 변환
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

# 모델에 입력하고 로짓(logits) 계산
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

# 소프트맥스를 이용한 확률 계산
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)

# 예측된 레이블 출력
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [0]


In [ ]:
# 이어지는 문장 예시
# 이어지는 문장은 0  이어지지 않은 문장은 1
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요."

# 두 문장을 토크나이저로 변환
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

# 모델에 입력하고 로짓(logits) 계산
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

# 소프트맥스를 이용한 확률 계산
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)

# 예측된 레이블 출력
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [1]


# TPU

In [ ]:
import tensorflow as tf

# TPU 연결을 위한 설정
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()

# 클러스터에 연결
tf.config.experimental_connect_to_cluster(resolver)

# TPU 시스템 초기화
tf.tpu.experimental.initialize_tpu_system(resolver)


런타임 새션 다시시작

In [ ]:
# TPU 장치 목록 확인
import tensorflow as tf
print("TPU가 정상적으로 연결되었는지 확인: ", tf.config.list_logical_devices('TPU'))


TPU가 정상적으로 연결되었는지 확인:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [ ]:
import tensorflow as tf

try:
    # TPU 클러스터 자동 감지
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # 주소를 자동으로 감지합니다.
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU 연결 성공!")
except ValueError as e:
    print(f"TPU 연결 실패: {e}")


TPU 연결 성공!


# 모델 정의 및 컴파일(tpu)

In [ ]:

def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(256, 3, activation='relu'),
        tf.keras.layers.Flatten(),  # Conv2D 출력 크기를 평탄화
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # 클래스 수에 맞게 출력
    ])
    return model

  # TPU에서 학습할 수 있도록 모델을 정의하고 컴파일
strategy = tf.distribute.TPUStrategy(resolver)
with strategy.scope():
    model = create_model()
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

# 모델 요약
model.summary()


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 26, 26, 256)         │           2,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 24, 24, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 147456)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 256)                 │      37,748,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,375,818 (146.39 MB)

 Trainable params: 38,375,818 (146.39 MB)

 Non-trainable params: 0 (0.00 B)

# 데이터 셋 준비


In [ ]:
# 데이터셋 로드 (MNIST 예시)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1)).astype('float32') / 255

# 데이터셋을 분산처리 가능하도록 배치 처리
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(5)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(5)


In [ ]:
# 데이터에 NaN 값이 있는지 확인
for data,label in train_dataset:
    if tf.math.is_nan(data).numpy().any():
        print("데이터에 NaN 값이 있습니다.")
        break
    else:
        print("데이터에 NaN 값이 없습니다.")
    break


데이터에 NaN 값이 없습니다.


In [ ]:
# 학습률을 점진적으로 감소시키는 콜백을 사용
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-10,  # 초기 학습률
    decay_steps=1000,  # 얼마나 자주 학습률을 감소시킬지
    decay_rate=0.9,  # 학습률 감소 비율
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])


# 분산 학습

In [ ]:
# TPU에서 학습
# 학습률을 점진적으로 감소시키는 콜백을 사용
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-10,  # 초기 학습률
    decay_steps=1000,  # 얼마나 자주 학습률을 감소시킬지
    decay_rate=0.9,  # 학습률 감소 비율
    staircase=True)
with strategy.scope():
    # optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule,clipvalue=1.0)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5,clipvalue=1.0)

    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # from_logits=False로 수정
                  metrics=['sparse_categorical_accuracy'])

    model.fit(train_dataset, epochs=5, validation_data=test_dataset)


Epoch 1/5
  199/12000 ━━━━━━━━━━━━━━━━━━━━ 3:53 20ms/step - loss: nan - sparse_categorical_accuracy: nan

KeyboardInterrupt: 

# NAN 발생 이유
```
1.학습률이 너무 큼: 너무 큰 학습률로 인해 가중치가 비정상적으로 커지고, 그로 인해 값이 발산하여 NaN이 발생할 수 있습니다.
2.데이터 문제: 학습 데이터에 이상값이 있거나, NaN 값이 포함된 경우에도 이런 문제가 발생할 수 있습니다.
3.모델 초기화 문제: 가중치 초기화가 잘못되었을 경우에도 NaN이 발생할 수 있습니다.
4.손실 함수의 불안정성: 예를 들어, SparseCategoricalCrossentropy와 Softmax가 함께 잘못 사용되는 경우가 있을 수 있습니다.
```

In [ ]:
# 학습률을 점진적으로 감소시키는 콜백을 사용
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-10,  # 초기 학습률
    decay_steps=1000,  # 얼마나 자주 학습률을 감소시킬지
    decay_rate=0.9,  # 학습률 감소 비율
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])


In [ ]:
# 데이터에 NaN 값이 있는지 확인
for data,label in train_dataset:
    if tf.math.is_nan(data).numpy().any():
        print("데이터에 NaN 값이 있습니다.")
        break
    else:
        print("데이터에 NaN 값이 없습니다.")
    break


데이터에 NaN 값이 없습니다.
